In [1]:
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
from sklearn.linear_model import LinearRegression

%matplotlib inline

# Create Training Data

Run the prep_train_data.py file to generate fitting data.
```
    python prep_train_data.py --n {n0} --cnum {cuda_num}
    python prep_train_data.py --n {n1} --cnum {cuda_num}
```

# Load Data

In [4]:
def get_qs(reserrlog, otlog, accs):

    qsreserrlog = []
    for i in range(len(reserrlog)):
        cur_arr = np.array(reserrlog[i])
        len_cur_arr = len(cur_arr)
    #     print(cur_arr)
    #     print(len_cur_arr)
        if len_cur_arr:
            len_to_add = 11 - len_cur_arr
            if len_to_add:
                to_add = np.zeros(len_to_add) + 0
                cur_arr = np.concatenate((cur_arr, to_add))
            qsreserrlog.append(cur_arr)


    #         print(len(cur_arr))
    qsreserrlog = np.array(qsreserrlog)

    qsotlog = []
    for i in range(len(otlog)):
        cur_arr = np.array(otlog[i])
        len_cur_arr = len(cur_arr)
        if len_cur_arr:
            len_to_add = 11 - len_cur_arr
            if len_to_add:
                to_add = np.zeros(len_to_add) + 0
                cur_arr = np.concatenate((cur_arr, to_add))
            qsotlog.append(cur_arr)

    qsaccs = []
    for i in range(len(accs)):
        cur_arr = np.array(accs[i])
        len_cur_arr = len(cur_arr)
        if len_cur_arr:
            len_to_add = 11 - len_cur_arr
            if len_to_add:
                to_add = np.zeros(len_to_add) + 0
                cur_arr = np.concatenate((cur_arr, to_add))
            qsaccs.append(cur_arr)

    #         print(len(cur_arr))
    qsotlog = np.array(qsotlog)
    qsaccs = np.array(qsaccs)
    
    return qsreserrlog, qsotlog, qsaccs



In [ ]:
size = 1000
reserrlog, otlog, accs = pickle.load(open(f"projektor_data/cif10_3sources_unbalanced_{size}_br_10_rep_5.res", 'rb'))
qsreserrlog10, qsotlog10, qsaccs10 = get_qs(reserrlog, otlog, accs) 

In [ ]:
size = 1500
reserrlog, otlog, accs = pickle.load(open(f"projektor_data/cif10_3sources_unbalanced_{size}_br_10_rep_5.res", 'rb'))
qsreserrlog15, qsotlog15, qsaccs15 = get_qs(reserrlog, otlog, accs) 

# Fit Data

## Create Mask for Test Data

In [ ]:
# q1 mask
size1 = qsreserrlog.shape[0]
size2 = qsreserrlog.shape[1]
q1mask1 = np.zeros([size1,size1])
q1mask2 = np.zeros([size1,size1])
thr = 5
for l1 in range(size1):
    for l2 in range(size1):
        if qsreserrlog[l1,l2] != 0:
            if l2 < thr:
                q1mask1[l1, l2] = 1
            if l1+l2 > 10:
                q1mask1[l1, l2] = 0
            if l2 >= thr:
                q1mask2[l1, l2] = 1
            if l1+l2 > 10:
                q1mask2[l1, l2] = 0
            
q1_mask_train_inds = q1mask1
q1_mask_pred_inds = q1mask2

q1mask_train = np.nonzero(q1_mask_train_inds != 0)
q1mask_pred = np.nonzero(q1_mask_pred_inds != 0)

In [ ]:
A1 = np.zeros((size1,size2))
A2 = np.zeros((size1,size2))

# Fill each 1-dimension subarray with [0, 1, 2, 3, 4, 5]
for i1 in range(size1):
    for i2 in range(size2):
        A1[i1, i2] = i1/(size1-1)
        A2[i1, i2] = i2/(size1-1)
print(A2)


## Fit Data with $\texttt{Projektor}$ for performance prediction for extrapolation
$$\hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N, \mathbf{p})),D^\text{val})=\sum_{i=1}^m (b_2^i\cdot p_i^2+b_1^i\cdot p_i+b_0)\cdot \text{OT}(\mathcal{D}(N, \mathbf{p}), D^\text{val})+\sum_{i=1}^m (c_2^i\cdot p_i^2+c_1^i\cdot p_i+c_0)$$

In [ ]:
from sklearn.linear_model import LinearRegression
npqsres = np.nan_to_num(np.array(qsreserrlog))
npqsaccs = np.nan_to_num(np.array(qsaccs))
npqsop = np.nan_to_num(np.array(qsotlog))

# mask q1mask2 set to 0 q1mask2/q1mask_pred
npqsres[q1mask_pred] = 0
npqsop[q1mask_pred] = 0
npqsaccs[q1mask_pred] = 0

# npqsres[cover_train_mask] = 0

res_non_zero_indices = np.nonzero(npqsres != 0)
accs_non_zero_indices = np.nonzero(npqsaccs != 0)
ot_non_zero_indices = np.nonzero(npqsop != 0)
# Extract the non-zero elements
res_non_zero_elements = np.log(npqsres[res_non_zero_indices])
accs_non_zero_elements = npqsaccs[ot_non_zero_indices]
ot_non_zero_elements = npqsop[ot_non_zero_indices]

scalingmat1 = A1
scalingmat2 = A2
scalingmat3 = 1 - scalingmat1 - scalingmat2 

qss1 = scalingmat1[ot_non_zero_indices]
qss2 = scalingmat2[ot_non_zero_indices]
qss3 = scalingmat3[ot_non_zero_indices]


y2fit = np.array(accs_non_zero_elements.reshape(-1, 1))
x2fit = np.zeros([np.shape(accs_non_zero_indices)[1], 8])
x2fit[:, 0] = ot_non_zero_elements
x2fit[:, 1] = qss1
x2fit[:, 2] = qss2
x2fit[:, 3] = qss1*ot_non_zero_elements
x2fit[:, 4] = qss2*ot_non_zero_elements
x2fit[:, 5] = qss2*qss2*ot_non_zero_elements
x2fit[:, 6] = qss1*qss1*ot_non_zero_elements

model = LinearRegression()
model.fit(x2fit,y2fit)

model_predicted_train = model.predict(x2fit).flatten()
y_train = y2fit.flatten()

## Predict Performance of Unseen $\textbf{p}$

In [ ]:
npqsres = np.nan_to_num(np.array(qsreserrlog))
npqsaccs = np.nan_to_num(np.array(qsaccs))
npqsop = np.nan_to_num(np.array(qsotlog))

# mask q1mask2 set to 0 q1mask2/q1mask_pred
# npqsres[q1mask_pred] = 0
# npqsop[q1mask_pred] = 0
# npqsaccs[q1mask_pred] = 0

# npqsres[cover_train_mask] = 0

res_non_zero_indices = np.nonzero(npqsres != 0)
accs_non_zero_indices = np.nonzero(npqsaccs != 0)
ot_non_zero_indices = np.nonzero(npqsop != 0)
# Extract the non-zero elements
res_non_zero_elements = np.log(npqsres[res_non_zero_indices])
accs_non_zero_elements = npqsaccs[ot_non_zero_indices]
ot_non_zero_elements = npqsop[ot_non_zero_indices]

scalingmat1 = A1
scalingmat2 = A2
scalingmat3 = 1 - scalingmat1 - scalingmat2 

qss1 = scalingmat1[ot_non_zero_indices]
qss2 = scalingmat2[ot_non_zero_indices]
qss3 = scalingmat3[ot_non_zero_indices]

y2fit = np.array(accs_non_zero_elements.reshape(-1, 1))
x2fit = np.zeros([np.shape(accs_non_zero_indices)[1], 8])
x2fit[:, 0] = ot_non_zero_elements
x2fit[:, 1] = qss1
x2fit[:, 2] = qss2
x2fit[:, 3] = qss1*ot_non_zero_elements
x2fit[:, 4] = qss2*ot_non_zero_elements
x2fit[:, 5] = qss2*qss2*ot_non_zero_elements
x2fit[:, 6] = qss1*qss1*ot_non_zero_elements

otqfit = model.predict(x2fit)
print(model.coef_)
resot = np.zeros([size1,size1])
resot[ot_non_zero_indices] = otqfit.flatten()

model_predicted_test = resot[q1mask_pred]
y_test = npqsaccs[q1mask_pred] # np.log(npqsres[q1mask_pred])

# Plot Fitted Data and Performance Prediction

In [ ]:
plt.scatter(model_predicted_train, y_train, marker='o', label='Training', color='blue', alpha=0.7)
plt.scatter(model_predicted_test, y_test, marker='o', label='Prediction (PQ-OT)', color='orange', alpha=0.7)
# plt.xlabel('pseudo-quadratic OT')
plt.xlabel('pseudo-quadratic OT prediction')
plt.ylabel('model err (log)')

# calculate the slope and intercept of the linear regression line
pltslope, pltintercept =[1, 0]
# create a line based on the linear regression equation
x_line = np.linspace(0, 4, 100)
y_line = pltslope * x_line + pltintercept
# plot the line
plt.plot(x_line, y_line, c='orange')
# plt.xlim([1, 3])
# plt.ylim([1, 3])
# plt.xlim([-3, 0.5])
# plt.ylim([-3, 0.5])

# show the plot
plt.legend()
# plt.title('Interpolation on Cifar 10 to Smaller Values by Centering Scaling')
plt.title(f'Extrapolation on DataSource#3(>{thr/size1 * 100:.0f}) by PQ-OT')
plt.show()

Y_train = y_train
Y_predict_train = model_predicted_train
mse_train = mean_squared_error(Y_train, Y_predict_train)

print("Train MSE: ", mse_train)

train_model_mean_perf_diff = mean_absolute_error(Y_predict_train - Y_train)
print("Train performance mean difference: ", train_model_mean_perf_diff)


print("--------TEST-----------")

Y_test = y_test
Y_predicted = model_predicted_test
mse_pred = mean_squared_error(Y_test, Y_predicted)
test_model_mean_perf_diff = mean_absolute_error(Y_predicted - Y_test)
print("Test performance mean difference: ", test_model_mean_perf_diff)

print("R2: ", r2_score(Y_test, Y_predicted))
print("MAE:", mean_absolute_error(Y_test, Y_predicted))
print("MSE: ", mean_squared_error(Y_test, Y_predicted))
print("RMSE: ", math.sqrt(mean_squared_error(Y_test, Y_predicted)))
# pqot_mse.append(mean_squared_error(Y_test, Y_predicted))


# Performance Projection

Fit above function for two datasets of size $N_0$ and $N_1$.
Then, project performance onto large data scales, using the equation below:

$$\hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N,\mathbf{p}); D^\text{val}) = \left(\log\frac{N_1}{N_0}\right)^{-1}\left[ \log\frac{ N}{N_0}\hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N_1,\mathbf{p})); D^\text{val})-\log\frac{ N}{N_1}\hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N_0,\mathbf{p}); D^\text{val})\right]$$

### Example for projecting performance onto N from $N_0$ and $N_1$

In [ ]:
n = 10000
n0 = 1000
acc0 = A # Accuracy of a model trained on N_0 for a given p

n1 = 1500
acc1 = B # Accuracy of a model trained on N_1 for a given p

# acc_10k = np.log(n1/n0) * (n/n0* acc1 -n/n1 * acc0)
acc_10k =  (1/np.log(n1/n0)) * (np.log(n/n0)* acc1 - np.log(n/n1) * acc0)

print(acc_10k)

#### To verify projection performance, we can actually train a model on $N$ points and have a direct comparison.
#### To do so, we can generate data points for size $N$:
    
```
    python prep_train_data.py --n {N} --cnum {cuda_num}
```

# Optimal Data Source Selection

We proceed to optimal data source selection after we have fitted function for data scales $N_0$ and $N_1$. To find the optimal mixing ratio $\textbf{p}^*$, we adopt a gradient-based method with each update as follows:

$$\mathbf{p^{t+1}}\leftarrow\mathbf{p^{t}} + d^t\cdot \left.\frac{\partial \hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N_s,\mathbf{p})),D^\text{val})}{\partial \mathbf{p}}\right|_{\mathbf{p}=\mathbf{p}^t},$$

where

$$\frac{\partial \hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N,\mathbf{p}); D^\text{val})}{\partial \mathbf{p}}=
     \left(\log\frac{N_1}{N_0}\right)^{-1}\left[ \log\frac{ N}{N_0}\frac{\partial \hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N_1,\mathbf{p})); D^\text{val})}{\partial \mathbf{p}}-\log\frac{ N}{N_1}\frac{\partial\hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N_0,\mathbf{p}); D^\text{val})}{\partial \mathbf{p}}\right].$$
     
Each gradient of $\hat{\mathcal{L}}$ can be defined as below:

$$\frac{\partial \hat{\mathcal{L}}(\mathcal{A}(\mathcal{D}(N,\mathbf{p}); D^\text{val})}{\partial p_i} =  (b_2^i\cdot p_i^2+b_1^i\cdot p_i+b_0)\cdot\frac{\partial \text{OT}(\mathcal{A}(\mathcal{D}(N,\mathbf{p}); D^\text{val})}{\partial p_i} 
    +[b_2^i\cdot (2p_i)+b_1^i]\cdot \text{OT}(\mathcal{A}(\mathcal{D}(N,\mathbf{p}); D^\text{val}) + [c_2^i\cdot (2p_i)+c_1^i]$$
    
and each gradient of OT can be efficiently computed as follows:

$$\frac{\partial \text{OT}(\mathcal{A}(\mathcal{D}(N,\mathbf{p}); D^\text{val})}{\partial p
    _i}=\frac{1}{n_i}\left(\sum_{j=1}^{n_i} f_i^j - \frac{n_i}{N-n_i}\sum_{x=\{1...m\}\setminus i}\sum_{y=1}^{n_x} f_x^y\right).$$

### We provide a full code for the optimization step below (whereas all helper functions are provided in the prep_train_data.py file.
This code follows the above example for $N=10,000$ and data scales $N_0 = 1,000$ and $N_1 = 1,500$.

In [ ]:

n = 10000
n0 = 1000
n1 = 1500
# q = 0.0
batch_size = 256


# make test dataloader

test_loader = torch.utils.data.DataLoader(dataset=TensorDataset(torch.Tensor(test_features).permute(0,3,1,2), torch.LongTensor(test_labels)), 
                                                batch_size=batch_size, 
                                                shuffle=False)

gdreserrlog = []
gdresacc = []
gdotlog = []
gd1log = []
gd2log = []

q1 = 1/3
q2 = 1/3
q3 = 1-q1-q2

q1log = [q1]
q2log = [q2]

lr = 5e-4
reps = 1
iters = 100
all_qs = []

print("iters: ", iters)
print("reps: ", reps)
for iteration in range(iters):
    cacheerr = []
    cacheacc = []
    cacheot = []
    cachegd1 = []
    cachegd2 = []
    cacheot_n1 = []
    cachegd1_n1 = []
    cachegd2_n1 = []
    
    start_t = time.time()
    all_qs.append((q1,q2,q3))

    for rep in range(reps):

        # create dataset
        train_x, train_y, ds1_len, ds2_len, ds3_len = dataset_q(q1, q2, n0, train_features, train_labels)
        # make train dataloader
        train_loader, train_loader_ot, test_loader = model_loaders(train_x, train_y,  test_features, test_labels, n0,transform)

        
        print(f"Lengths: S1={ds1_len}, S2={ds2_len}, S3={ds3_len}")
        # get OT dist
        ot_dist, dual_sol = get_ot_dist(train_loader_ot, test_loader, n=n0)
        cacheot.append(ot_dist)

        # create dataset
        train_x_n1, train_y_n1, ds1_len_n1, ds2_len_n1, ds3_len_n1 = dataset_q(q1, q2, n1, train_features, train_labels)
        # make train dataloader
        train_loader_n1, train_loader_ot_n1, test_loader_n1 = model_loaders(train_x_n1, train_y_n1,  test_features, test_labels, n1, transform)

        
        print(f"_n1 Lengths: S1={ds1_len_n1}, S2={ds2_len_n1}, S3={ds3_len_n1}")
        # get OT dist
        ot_dist_n1, dual_sol_n1 = get_ot_dist(train_loader_ot_n1, test_loader_n1, n=n1)
        cacheot_n1.append(ot_dist_n1)
    
    
        train_x_n, train_y_n, _, _, _ = dataset_q(q1, q2, n, train_features, train_labels)
        # make train dataloader
        train_loader_n, train_loader_ot_n, test_loader_n = model_loaders(train_x_n, train_y_n,  test_features, test_labels, n,transform)

    
#         get model error (Optional: Can be commented our for faster optimization)
        errloss, acc = get_model_log_err(train_loader_n, test_loader_n)
        cacheerr.append(errloss)
        cacheacc.append(acc)
#         cacheerr.append(0) #get_model_log_err(train_loader, test_loader))
        
        
        full_ds_len = np.shape(train_x)[0]
        
        full_ds_len_n1 = np.shape(train_x_n1)[0]
        
        
        q1_end_pos = ds1_len
        q2_end_pos = q1_end_pos + ds2_len
        q3_end_pos = q2_end_pos + ds3_len
        
        q1_end_pos_n1 = ds1_len_n1
        q2_end_pos_n1 = q1_end_pos_n1 + ds2_len_n1
        q3_end_pos_n1 = q2_end_pos_n1 + ds3_len_n1
        
        fall = dual_sol
        fds1= fall[0:q1_end_pos]
        fds2 = fall[q1_end_pos:q2_end_pos]
        fds3 = fall[q2_end_pos:q3_end_pos]
        
        fall_n1 = dual_sol_n1
        fds1_n1= fall_n1[0:q1_end_pos_n1]
        fds2_n1 = fall_n1[q1_end_pos_n1:q2_end_pos_n1]
        fds3_n1 = fall_n1[q2_end_pos_n1:q3_end_pos_n1]
        
        # Calibrated Gradient for Q1
        if full_ds_len-ds1_len != 0:
            calib_q1 = np.sum(fds1) - (np.sum(fall) - np.sum(fds1))*(ds1_len/(full_ds_len-ds1_len))
        else:
            calib_q1 = 0

        if full_ds_len_n1-ds1_len_n1 != 0:
            calib_q1_n1 = np.sum(fds1_n1) - (np.sum(fall_n1) - np.sum(fds1_n1))*(ds1_len_n1/(full_ds_len_n1-ds1_len_n1))
        else:
            calib_q1_n1 = 0
            
        # Calibrated Gradient for Q2    
        if full_ds_len-ds2_len != 0:
            calib_q2 = np.sum(fds2) - (np.sum(fall) - np.sum(fds2))*(ds2_len/(full_ds_len-ds2_len))
        else:
            calib_q2 = 0
            
            
        if full_ds_len_n1-ds2_len_n1 != 0:
            calib_q2_n1 = np.sum(fds2_n1) - (np.sum(fall_n1) - np.sum(fds2_n1))*(ds2_len_n1/(full_ds_len_n1-ds2_len_n1))
        else:
            calib_q2_n1 = 0
            
        # Gradient for OT for Q1
        if ds1_len != 0:
            calib_q1 /= ds1_len
            gd_q1 = (b2q1*q1*q1+b1q1*q1+b0)*calib_q1+(b2q1*2*q1+b1q1)*ot_dist+(c2q1*2*q1+c1q1)
        else:
            gd_q1 = 0
            
           
        if ds1_len_n1 != 0:
            calib_q1_n1 /= ds1_len_n1
            gd_q1_n1 = (b2q1_n1*q1*q1+b1q1_n1*q1+b0_n1) *calib_q1_n1+(b2q1_n1*2*q1+b1q1_n1)*ot_dist_n1+(c2q1_n1*2*q1+c1q1_n1)
        else:
            gd_q1_n1 = 0
        
        # Gradient for OT for Q2
        if ds2_len != 0:
            calib_q2 /= ds2_len
            gd_q2 = (b2q2*q2*q2+b1q2*q2+b0)*calib_q2+(b2q2*2*q2+b1q2)*ot_dist+(c2q2*2*q2+c1q2)
        
            
        if ds2_len_n1 != 0:
            calib_q2_n1 /= ds2_len_n1
            gd_q2_n1 = (b2q2_n1*q2*q2+b1q2_n1*q2+b0_n1) *calib_q2_n1+(b2q2_n1*2*q2+b1q2_n1)*ot_dist_n1+(c2q2_n1*2*q2+c1q2_n1)
        
        
        if (gd_q1 == 0) & (gd_q2 == 0):
            print("zero gradient!")
            break
        if (gd_q1_n1 == 0) & (gd_q2_n1 == 0):
            print("zero gradient!")
            break
        cachegd1.append(gd_q1)
        cachegd2.append(gd_q2)
        cachegd1_n1.append(gd_q1_n1)
        cachegd2_n1.append(gd_q2_n1)
        
    term1 = 1/np.log(n1/n0)
    term2 = np.log(n/n0)
    term3 = np.log(n/n1)
    
    grad1 = term1 * (term2*np.median(cachegd1_n1) - term3*np.median(cachegd1))
    grad2 = term1 * (term2*np.median(cachegd2_n1) - term3*np.median(cachegd2))
    step = lr*np.power(0.90, iteration)
    q1 = q1 + gd_q1*step
    q2 = q2 + gd_q2*step
    q3 = 1-q1-q2
    q1log.append(q1)
    q2log.append(q2)
    gdreserrlog.append(np.median(cacheerr))
    gdotlog.append(np.median(cacheot))
    gdresacc.append(np.median(cacheacc))
    gd1log.append(grad1)
    gd2log.append(grad2)
    print(np.median(cacheot))
    print("cacheerr: ", cacheerr)
    print("cacheot: ", cacheot)
    print("cacheacc: ", cacheacc)   
    print("iter: ", iteration, " it took: ", time.time() - start_t)
    
    print("\n New ITER")
    print("GRADIENT: ", gd_q1, gd_q2, step)
    print(f'Q1 {q1}, Q2 {q2}, Q3 {q3}')
    
